In [155]:
%matplotlib widget
import os
import re
os.environ["GDAL_DATA"] = "/home/parndt/anaconda3/envs/geo_py37/share/gdal"
os.environ["PROJ_LIB"] = "/Users/parndt/anaconda3/envs/geo_py37/share/proj"
os.environ["PROJ_DATA"] = "/Users/parndt/anaconda3/envs/geo_py37/share/proj"
import h5py
import math
import datetime
import traceback
import shapely
import requests
import zipfile
import shutil
import pytz
from tzwhere import tzwhere
from dateutil import tz
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec
from cmcrameri import cm as cmc
from mpl_toolkits.axes_grid1 import make_axes_locatable
# from icelakes.utilities import convert_time_to_string
from IPython.display import Image, display
from matplotlib.collections import PatchCollection
from sklearn.neighbors import KDTree
import scipy as sc
from scipy.interpolate import RectBivariateSpline
from scipy.interpolate import RegularGridInterpolator
from scipy import signal
from scipy.stats import binned_statistic
from scipy.signal import find_peaks
import hdbscan
import rasterio as rio
from rasterio import plot as rioplot
from rasterio import warp
from rasterio import Affine as A
from rasterio.enums import ColorInterp
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.transform import TransformMethodsMixin
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.crs import CRS as rioCRS
from ipyleaflet import Map, basemaps, Polygon, GeoData, LayersControl, Polyline, Marker
from geopy.distance import geodesic
from utils import read_atl03
from utils import intersection
from utils import download_is2
from ed.edcreds import getedcreds
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

def get_signal(df, n_iter=100, h_thresh=[200,3], xatc_win=[500,7]):
    number_iterations = n_iter
    elevation_threshold_start = h_thresh[0]
    elevation_threshold_end = h_thresh[1]
    xatc_window_width_start = xatc_win[0]
    xatc_window_width_end = xatc_win[1]
    
    df.xatc -= df.xatc.min()
    df.xatc + np.random.uniform(low=-0.35, high=0.35, size=len(df))
    df['is_signal'] = True
    df['rmean'] = np.nan
    
    def get_params(start,end,number):
        return [start * ((end / start)**(1/(number-1))) ** it for it in range(number)]
    
    h_diff_thresholds = get_params(start=elevation_threshold_start, end=elevation_threshold_end, number=number_iterations)
    window_sizes = np.int32(np.round(get_params(start=xatc_window_width_start, end=xatc_window_width_end, number=number_iterations)))

    for i in range(number_iterations):
        rmean = df.h[df.is_signal].rolling(3*window_sizes[i],center=True,min_periods=5,win_type='gaussian').mean(
            std=int(np.ceil(window_sizes[i]/2)))
        df['rmean'] = np.interp(df.xatc, df.xatc[df.is_signal], rmean)
        df['is_signal'] = np.abs(df.rmean-df.h) < h_diff_thresholds[i]
        
    rmean = df.h[df.is_signal].rolling(5*window_sizes[i],center=True,min_periods=5,win_type='gaussian').mean(std=window_sizes[-1])
    df['rmean'] = np.interp(df.xatc, df.xatc[df.is_signal], rmean)
    
    return df

def get_oa_url(df, ancillary, gtx, verbose=False):
    datenum = ancillary['granule_id'][6:14]
    date = '%s-%s-%s' % (datenum[:4], datenum[4:6], datenum[6:])
    lonmin = df.lon.min()
    latmin = df.lat.min()
    lonmax = df.lon.max()
    latmax = df.lat.max()
    rgt = ancillary['rgt']
    oa_base_url = 'http://openaltimetry.org/data/api/icesat2/atl03?'
    oa_param_str = 'date={date:s}&minx={lonmin:.7f}&miny={latmin:.7f}&maxx={lonmax:.7f}&maxy={latmax:.7f}&trackId={rgt:d}&beamName={gtx:s}'
    oa_output = '&outputFormat=json'
    oa_param_str = oa_param_str.format(date=date,lonmin=lonmin,latmin=latmin,lonmax=lonmax,latmax=latmax,rgt=rgt,gtx=gtx)
    oa_url = oa_base_url + oa_param_str + oa_output
    ancillary['date'] = date
    ancillary['oa_url'] = oa_url
    if verbose: print(oa_url)
    return ancillary

def get_pulseid_and_gt_latlon(df):
    df['pulseid'] = 1000*df.mframe.astype(np.uint64)+df.ph_id_pulse.astype(np.uint64)
    df['ph_index'] = df.index
    df = df.set_index('pulseid')
    thegroup = df.groupby('pulseid')
    df_grouped = thegroup[['xatc', 'lat', 'lon', 'rmean', 'h']].median()
    return df_grouped

def get_along_track_elev(df, df_grouped, xatc_resolution=0.7): 
    xatc = np.arange(df.xatc[df.is_signal].min(), df.xatc[df.is_signal].max(), xatc_resolution)
    elev = np.interp(xatc, df_grouped.xatc, df_grouped.rmean)
    lat = np.interp(xatc, df_grouped.xatc, df_grouped.lat)
    lon = np.interp(xatc, df_grouped.xatc, df_grouped.lon)
    df_out = pd.DataFrame({'lat':lat, 'lon':lon, 'xatc':xatc, 'elev_is2':elev})
    return df_out

def interpolate_dtm(terrain_model_file, elevs): 
    dtm = rio.open(terrain_model_file)
    gt_x, gt_y = warp.transform({'init': 'epsg:4326'}, dtm.crs, elevs.lon, elevs.lat)
    elevs['gt_x'] = gt_x
    elevs['gt_y'] = gt_y
    dtm_data = dtm.read(1)
    dtm_data[dtm_data==-9999] = np.nan
    _, ys = rio.transform.xy(dtm.transform, np.arange(dtm.height)+0.5, np.zeros(dtm.height))
    xs, _ = rio.transform.xy(dtm.transform, np.zeros(dtm.width), np.arange(dtm.width)+0.5)
    xs = np.array(xs)
    ys = np.flip(np.array(ys))
    X,Y = np.meshgrid(xs,ys)
    dtm_data = np.flipud(dtm_data)
    interpolator = RegularGridInterpolator((ys,xs), dtm_data, method='linear', bounds_error=False, fill_value=np.nan)
    dtm_elev_interp = interpolator((gt_y, gt_x))
    elevs['elev_dtm'] = dtm_elev_interp
    elevs = elevs[~np.isnan(elevs.elev_dtm)].copy()
    elevs['elev_diff'] = elevs.elev_is2 - elevs.elev_dtm 
    return elevs

def latlonelev2xyz(lat, lon, elevation, equatorial_radius = 6378137, flattening = 1/298.257223563): 
    # point is (lat, lon), elevation is elevation above reference ellipsoid in meters
    # equatorial_radius (in meters) and flattening correspond to the reference ellipsoid (default: WGS84)
    # implemented according to https://gssc.esa.int/navipedia/index.php/Ellipsoidal_and_Cartesian_Coordinates_Conversion
    phi = lat * (np.pi / 180)
    lamda = lon * (np.pi / 180)
    h = elevation
    f = flattening
    a = equatorial_radius
    e_squared = 2*f - f**2
    N = a / np.sqrt(1 - e_squared * np.sin(phi)**2)
    x = (N + h) * np.cos(phi) * np.cos(lamda)
    y = (N + h) * np.cos(phi) * np.sin(lamda)
    z = ((1-e_squared) * N + h) * np.sin(phi)
    return x, y, z


def xyz2latlonelev(x, y, z, equatorial_radius = 6378137, flattening = 1/298.257223563, h_tol=1e-3, lat_tol_rad=1e-6, max_iter=1e6): 
    # x, y, z are in ECEF cartesian coordinates, in meters
    # equatorial_radius (in meters) and flattening correspond to the reference ellipsoid (default: WGS84)
    # implemented according to https://gssc.esa.int/navipedia/index.php/Ellipsoidal_and_Cartesian_Coordinates_Conversion
    f = flattening
    a = equatorial_radius
    e_squared = 2*f - f**2
    p = np.sqrt(x**2 + y**2)
    lon_rad = np.arctan2(y,x)
    lon = lon_rad * (180 / np.pi)
    phi_old = np.arctan2(z, (1-e_squared)*p)
    h_old = -1e10
    h_converged = False
    phi_converged = False
    i = 0
    while not (h_converged & phi_converged): 
        N_i = a / np.sqrt(1 - e_squared * np.sin(phi_old)**2)
        h_new = p / np.cos(phi_old) - N_i
        phi_new = np.arctan2(z, ( 1 - e_squared * ( N_i / ( N_i+h_new ) ) ) * p)
        h_converged = np.abs(h_new - h_old) < h_tol
        phi_converged = np.abs(phi_new - phi_old) < lat_tol_rad
        h_old = h_new
        phi_old = phi_new
        if i > max_iter: 
            break;
        i += 1
    elev = h_new
    lat = phi_new * (180 / np.pi)
    return lat, lon, elev


def get_distance_direct(lat1, lon1, elevation1, lat2, lon2, elevation2, equatorial_radius = 6378137, flattening = 1/298.257223563):
    # point1 and point2 are (lat, lon), elevation is elevation above reference ellipsoid in meters
    x1, y1, z1 = latlonelev2xyz(lat1, lon1, elevation1, equatorial_radius, flattening)
    x2, y2, z2 = latlonelev2xyz(lat2, lon2, elevation2, equatorial_radius, flattening)

    dist = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return dist, (x1, y1, z1), (x2, y2, z2)


def get_elev(latlon, terrain_model_file=None, dtm=None, dtm_data=None):
    # get elevation of the terrain model at the lat/lon coordinates of the target point
    if (terrain_model_file is None) & ((dtm is None) | (dtm_data is None)):
        print('ERROR: No terrain model file or rasterio reader + data supplied')
        return
    if terrain_model_file is not None:
        dtm = rio.open(terrain_model_file)
        dtm_data = dtm.read(1)
    gt_x, gt_y = warp.transform({'init': 'epsg:4326'}, dtm.crs, [latlon[1]], [latlon[0]])
    dtm_data[dtm_data==-9999] = np.nan
    _, ys = rio.transform.xy(dtm.transform, np.arange(dtm.height)+0.5, np.zeros(dtm.height))
    xs, _ = rio.transform.xy(dtm.transform, np.zeros(dtm.width), np.arange(dtm.width)+0.5)
    xs = np.array(xs)
    ys = np.flip(np.array(ys))
    X,Y = np.meshgrid(xs,ys)
    dtm_data = np.flipud(dtm_data)
    interpolator = RegularGridInterpolator((ys,xs), dtm_data, method='linear', bounds_error=False, fill_value=np.nan)
    elev = interpolator((gt_y, gt_x))[0]
    return elev


def get_rgt(latlon_target, kml_filename):
    # trim the kml to a rough bounding box around the TOO coords
    latlims = [latlon_target[0]-1, latlon_target[0]+1]
    lonlims = [latlon_target[1]-1, latlon_target[1]+1]

    with open(kml_filename, 'r') as file:
        kml_string = file.read()
    kml_string = kml_string[kml_string.find('LineString_kml'):kml_string.find('</LineString>')]
    kml_coords_str = '[[' + kml_string[kml_string.find('<coordinates>')+len('<coordinates>'):kml_string.find('</coordinates>')-1] + ']]'
    kml_coords_str = kml_coords_str.replace(' ', '],[')
    kml_coords_array = np.array(eval(kml_coords_str))

    kml_lon = kml_coords_array[:,0]
    kml_lat = kml_coords_array[:,1]
    is_in_bounds_lon = (kml_lon >= lonlims[0]) & (kml_lon <= lonlims[1])
    is_in_bounds_lat = (kml_lat >= latlims[0]) & (kml_lat <= latlims[1])
    is_in_bounds = is_in_bounds_lon & is_in_bounds_lat
    kml_lon = kml_lon[is_in_bounds]
    kml_lat = kml_lat[is_in_bounds]
    return kml_lon, kml_lat


def get_closest_points_kml(kml_lat, kml_lon, too_latlon, too_elev):
    diff_df = pd.DataFrame({'lat': kml_lat, 'lon': kml_lon, 'elev': too_elev, 'dist': np.nan, 
                            'x': np.nan, 'y': np.nan, 'z': np.nan, 'idx': np.arange(len(kml_lat))})
    for i in range(len(diff_df)):
        pt = diff_df.iloc[i]
        dist, xyz1, xyz2 = get_distance_direct(lat1=pt.lat, lon1=pt.lon, elevation1=pt.elev, 
                                               lat2=too_latlon[0], lon2=too_latlon[1], elevation2=pt.elev)
        diff_df.loc[i,'dist'] = dist
        diff_df.loc[i,'x'] = xyz1[0]
        diff_df.loc[i,'y'] = xyz1[1]
        diff_df.loc[i,'z'] = xyz1[2]
    diff_df.sort_values(by='dist', inplace=True)
    diff_df = diff_df.iloc[:2].copy()
    diff_df.sort_values(by='idx', inplace=True)
    return diff_df


def get_closest_point(closest_points_kml_df, too_latlon, too_elev, n_pts=10000):
    n_pts = 10000
    lats_ = np.linspace(closest_points_kml_df.iloc[0].lat, closest_points_kml_df.iloc[1].lat, n_pts)
    lons_ = np.linspace(closest_points_kml_df.iloc[0].lon, closest_points_kml_df.iloc[1].lon, n_pts)
    diff_df = pd.DataFrame({'lat': lats_, 'lon': lons_, 'elev': too_elev, 'dist': np.nan, 
                            'x': np.nan, 'y': np.nan, 'z': np.nan})
    for i in range(len(diff_df)):
        pt = diff_df.iloc[i]
        dist, xyz1, xyz2 = get_distance_direct(lat1=pt.lat, lon1=pt.lon, elevation1=pt.elev, 
                                               lat2=too_latlon[0], lon2=too_latlon[1], elevation2=pt.elev)
        diff_df.loc[i,'dist'] = dist
        diff_df.loc[i,'x'] = xyz1[0]
        diff_df.loc[i,'y'] = xyz1[1]
        diff_df.loc[i,'z'] = xyz1[2]
    diff_df.sort_values(by='dist', inplace=True)
    closest_point = diff_df.iloc[0].copy()
    return closest_point


def unit_vector(vector):
    return vector / np.linalg.norm(vector)


def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


def get_gt_on_terrain(start, direction, dtm, dtm_data, fit_to_zero=False, disp=False):
    
    def line_func(t):
        return start + t*direction
    
    def target_func(t):
        p_line = line_func(t)
        lat, lon, elev = xyz2latlonelev(*p_line, h_tol=1e-2, lat_tol_rad=1e-4)
        if fit_to_zero:
            elev_surface = 0.0
        else:
            elev_surface = get_elev((lat,lon), dtm=dtm, dtm_data=dtm_data)
            if np.isnan(elev_surface): 
                elev_surface = np.nanmean(dtm_data[dtm_data>-1000])
        elev_diff = elev-elev_surface
        return np.sum((elev - elev_surface)**2)
    
    t_opt = sc.optimize.fmin(target_func, x0=1.0, disp=disp);
    intersection_point = xyz2latlonelev(*line_func(t_opt))
    gt_x, gt_y = warp.transform({'init': 'epsg:4326'}, dtm.crs, [intersection_point[1]], [intersection_point[0]])
    return (intersection_point, (gt_x[0], gt_y[0]))


def add_timestamps(kml_df, kml_file, too_latlon, local_time_zone=None):
    with open(kml_file, 'r') as file:
            kml_string = file.read()
    kml_string.find('<Point id=')
    starts = [m.start() for m in re.finditer('<Point id=', kml_string)]
    ends = [m.start()+100 for m in re.finditer('</Point>', kml_string)]
    stringlist = [kml_string[s:e] for s,e in list(zip(starts, ends))]
    lats, lons, dts_utc, dts_local, dtstmp_utc, dtstmp_local, str_utc, str_local = [], [], [], [], [], [], [], []
    for substr in stringlist:
        coordstr = substr[substr.find('<coordinates>')+len('<coordinates>'):substr.find('</coordinates>')]
        coordstr = '['+coordstr[:coordstr.rfind(',')-1]+']'
        coords = eval(coordstr)
        lat = coords[1]
        lon = coords[0]
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        dt_str = descr[descr.find('DOY-'):]
        dt_str = dt_str[dt_str.find(' ')+1:]
        dt_str
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        dt_str = descr[descr.find('DOY-'):]
        dt_str = dt_str[dt_str.find(' ')+1:]
        day = int(dt_str[:dt_str.find('-')])
        month_abbr = dt_str[dt_str.find('-')+1:dt_str.rfind('-')]
        year = int(dt_str[dt_str.rfind('-')+1:dt_str.find(' ')])
        hrs = int(dt_str[dt_str.find(' ')+1:dt_str.find(':')])
        mins = int(dt_str[dt_str.find(':')+1:dt_str.rfind(':')])
        secs = int(dt_str[dt_str.rfind(':')+1:])
        datetime_str = '%4i-%3s-%02iT%02i:%02i:%02iZ' % (year, month_abbr, day, hrs, mins, secs)
        dt = datetime.datetime.strptime(datetime_str,'%Y-%b-%dT%H:%M:%SZ')
        timestamp_utc = datetime.datetime.timestamp(dt)
        from_zone = tz.gettz('UTC')
        to_zone = tz.gettz(local_time_zone)
        if local_time_zone is None:
            tzw = tzwhere.tzwhere()
            local_time_zone = tzw.tzNameAt(coords[1], coords[0])
            to_zone = tz.gettz(local_time_zone)
        dt_local = dt.astimezone(to_zone)
        timedelta_local = dt_local.utcoffset()
        lats.append(lat)
        lons.append(lon)
        dtstmp_utc.append(timestamp_utc)
    df = pd.DataFrame({'lat':lats, 'lon':lons, 'dtstmp_UTC': dtstmp_utc})
    near_lat = (df.lat >= too_latlon[0]-10) & (df.lat <= too_latlon[0]+10)
    near_lon = (df.lon >= too_latlon[1]-10) & (df.lon <= too_latlon[1]+10)
    df = df[near_lat & near_lon]
    dtstmp_utc_interp = np.interp(kml_df.lat, df.lat, df.dtstmp_UTC)
    kml_df['dtstmp_utc'] = dtstmp_utc_interp
    return kml_df, timedelta_local


# get the actual laser footprint tracks on the ground
def get_footprints(dtm, closest_points_kml_df, submission_point_dtm, spacecraft_orientation, ground_track_buffer,
                  ground_track_resolution, sc_closest_xyz, beam_strength, local_crs_meters, verbose=False):
    xs, ys = warp.transform({'init': 'epsg:4326'}, local_crs_meters, np.array(closest_points_kml_df.lon), np.array(closest_points_kml_df.lat))
    dx = xs[1] - xs[0]
    dy = ys[1] - ys[0]
    normal_vector = [-dy, dx] / np.sqrt(dy**2 + dx**2)
    x_left = submission_point_dtm[0] + normal_vector[0] * 45
    y_left = submission_point_dtm[1] + normal_vector[1] * 45
    x_right = submission_point_dtm[0] - normal_vector[0] * 45
    y_right = submission_point_dtm[1] - normal_vector[1] * 45
    lons, lats = warp.transform(local_crs_meters, {'init': 'epsg:4326'}, np.array([x_left,x_right]), np.array([y_left, y_right]))
    closest_points_beams_latlonelev = {'left': (lats[0], lons[0], 0.0), 'right': (lats[1], lons[1], 0.0)}
    closest_points_beams_xyzs = {k:np.array(latlonelev2xyz(*v)) for (k,v) in closest_points_beams_latlonelev.items()}
    p1_latlonelev = (closest_points_kml_df.iloc[0].lat, closest_points_kml_df.iloc[0].lon, closest_points_kml_df.iloc[0].elev)
    p2_latlonelev = (closest_points_kml_df.iloc[1].lat, closest_points_kml_df.iloc[1].lon, closest_points_kml_df.iloc[1].elev)
    p1_xyz = np.array(latlonelev2xyz(*p1_latlonelev))
    p2_xyz = np.array(latlonelev2xyz(*p2_latlonelev))
    flight_direction = unit_vector(p2_xyz - p1_xyz)
    sc_offsets_before = -np.arange(0, np.abs(ground_track_buffer[0])+1e-5, ground_track_resolution)
    sc_offsets_after = np.arange(0, np.abs(ground_track_buffer[1])+1e-5, ground_track_resolution)
    offsets_sc = np.sort(np.concatenate((sc_offsets_before, sc_offsets_after[1:])))
    sc_positions_xyz = [sc_closest_xyz + offset * flight_direction for offset in offsets_sc]
    footprints_dict = {}
    for side in beam_strength.keys():
        direction = closest_points_beams_xyzs[side] - sc_closest_xyz
        intersection_points, gt_xys_dtm_crs = [], []
        if verbose: print('Calculating laser footprint track on the topography (%s side, %s beam)' % (side, beam_strength[side]))
        for i, start_point in enumerate(sc_positions_xyz):
            intersection_point, gt_xy = get_gt_on_terrain(start_point, direction, dtm=dtm, dtm_data=dtm_data, fit_to_zero=False, disp=False)
            intersection_points.append(intersection_point)
            gt_xys_dtm_crs.append(gt_xy)
            if verbose: print('---> calculated %4i / %4i points' % (i+1,len(sc_positions_xyz)), end='\r')
        if verbose: print('---> DONE!                                                                        ')
        df_footprints = pd.DataFrame({'lat': np.array(intersection_points)[:,0],
                                      'lon': np.array(intersection_points)[:,1],
                                      'elev': np.array(intersection_points)[:,2],
                                      'rgt_x': np.array(gt_xys_dtm_crs)[:,0],
                                      'rgt_y': np.array(gt_xys_dtm_crs)[:,1],
                                     })
        footprints_dict[side] = df_footprints
    return footprints_dict

def plot_over_dtm(dtm_viz_file, too_latlon, submission_point_dtm, footprints_dict, beam_strength, returnfig=False):
    viz = rio.open(dtm_viz_file)
    fig, ax = plt.subplots(figsize=[7, 7], dpi=100)
    plt.rcParams.update({'font.size': 7})
    img_plot = rioplot.show(viz, ax=ax)
    submission_point_dtm = warp.transform({'init': 'epsg:4326'},viz.crs, [too_submit_latlonelev[1]],[too_submit_latlonelev[0]])
    ax.plot(submission_point_dtm[0], submission_point_dtm[1], 'bo', label='TOO submission point', ms=3)
    calc_surf_point_dtm = warp.transform({'init': 'epsg:4326'},viz.crs,[too_latlon[1]], [too_latlon[0]])
    ax.plot(calc_surf_point_dtm[0], calc_surf_point_dtm[1], 'ko', label='calculated actual point on surface', ms=3)
    for side in footprints_dict.keys():
        beamstrength = beam_strength[side]
        df_footprints = footprints_dict[side]
        ls = '-' if beamstrength=='strong' else '--'
        ax.plot(df_footprints.rgt_x, df_footprints.rgt_y, c='k', lw=0.5, marker='o', ms=1, ls=ls,
                label='TOO center %s beam\nfootprint track on topography' % beamstrength)
    for gtx in gtx_list:
        beamstrength = ancillary['gtx_strength_dict'][gtx]
        df = is2dict[gtx]
        ls = '-' if beamstrength=='strong' else '--'
        ax.plot(df.gt_x, df.gt_y, c='r', lw=0.5, ls=ls,
                    label='actual ICESat-2 locations (%s, %s)' % (gtx,beamstrength))
    ax.legend(loc='upper right')
    ax.set_xlim((viz.bounds.left, viz.bounds.right))
    ax.set_ylim((viz.bounds.bottom, viz.bounds.top))
    if returnfig:
        return fig

def plot_leaflet(kml_lat, kml_lon, too_latlon, too_submission_point, closest_point, footprints_dict, beam_strength):
    m=Map(basemap=basemaps.Esri.WorldImagery, center=too_latlon, zoom=11)
    line_loc_list = list(zip(kml_lat.T,kml_lon.T))
    line = Polyline(locations=line_loc_list,color="green",weight=3,fill=False)
    m.add_layer(line)
    marker_too = Marker(location=too_latlon, draggable=False, title='TOO target')
    m.add_layer(marker_too);
    marker_too_sub = Marker(location=too_submission_point, draggable=False, title='TOO submission point')
    m.add_layer(marker_too_sub);
    closest_point_plot = (closest_point.lat,closest_point.lon)
    marker_closest_point = Marker(location=closest_point_plot, draggable=False, opacity=0.7, title='closest point on RGT')
    m.add_layer(marker_closest_point);
    connector = Polyline(locations=[too_latlon, closest_point_plot],color="blue", weight=1, fill=False)
    m.add_layer(connector)
    for side in footprints_dict.keys():
        beamstrength = beam_strength[side]
        df_footprints = footprints_dict[side]
        thisweight = 2 if beamstrength=='strong' else 1
        line_loc_list = list(zip(df_footprints.lat.T,df_footprints.lon.T))
        thisline = Polyline(locations=line_loc_list,color="black",weight=thisweight,fill=False)
        m.add_layer(thisline)
    return m

def get_utm_crs_from_latlon(lat, lon):
    utm_crs_list = query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest=AreaOfInterest(west_lon_degree=lon,south_lat_degree=lat,east_lon_degree=lon,north_lat_degree=lat))
    utm_crs = CRS.from_epsg(utm_crs_list[0].code)
    return utm_crs

def get_sc_orient(thisdate):
    # from Tech Ref Table 20230307
    flipdates = ['01-Jan-2018/00:00:00', '28-Dec-2018/18:53:08', '07-Sep-2019/01:04:06', 
                 '14-May-2020/01:49:03', '15-Jan-2021/15:17:01', '02-Oct-2021/02:20:01', 
                 '09-Jun-2022/01:31:19', '09-Feb-2023/16:59:14', '01-Jan-2024/00:00:00']
    thisformat = '%d-%b-%Y/%H:%M:%S'
    flipdates_dt = [datetime.datetime.strptime(flpdt, thisformat) for flpdt in flipdates]
    sc_orients = np.array(['forward' if i%2==0 else 'backward' for i in range(len(flipdates)-1)])
    is_in_range = np.array([thisdate >= flpdt for flpdt in flipdates_dt[:-1]]) & np.array([thisdate < flpdt for flpdt in flipdates_dt[1:]])
    orient = sc_orients[is_in_range][0]
    return orient

def check_too(too_submit_latlonelev, kml_file_thiscycle, kml_file_RGTpointingcycle, atl03_file, 
              spacecraft_elevation, terrain_model_file, dtm_viz_file, local_time_zone, ground_track_buffer, ground_track_resolution):
    
    granule_name = atl03_file[atl03_file.find('processed_')+len('processed_'):]
    gtx_list = ['gt2l', 'gt2r']
    dtm = rio.open(terrain_model_file)
    dtm_data = dtm.read(1)
    local_crs_meters = get_utm_crs_from_latlon(*too_submit_latlonelev[:2])
    too_elev = get_elev(too_submit_latlonelev[:2], dtm=dtm, dtm_data=dtm_data)
    kml_lon, kml_lat = get_rgt(too_submit_latlonelev[:2], kml_file_RGTpointingcycle)
    kml_df = pd.DataFrame({'lon': kml_lon, 'lat': kml_lat})
    kml_df, timedelta_localtime = add_timestamps(kml_df, kml_file_thiscycle, too_submit_latlonelev[:2], local_time_zone=local_time_zone)
    closest_points_kml_df =  get_closest_points_kml(kml_lat, kml_lon, too_submit_latlonelev[:2], too_elev=0.0)
    closest_point = get_closest_point(closest_points_kml_df, too_submit_latlonelev[:2], too_elev=0.0, n_pts=100000)

    sc_closest_latlonelev = (closest_point.lat, closest_point.lon, spacecraft_elevation)
    sc_closest_xyz = np.array(latlonelev2xyz(*sc_closest_latlonelev))
    closest_point_rgt_xyz = np.array((closest_point.x, closest_point.y, closest_point.z))
    too_target_xyz = np.array(latlonelev2xyz(too_submit_latlonelev[0], too_submit_latlonelev[1], 0.0))
    off_point_angle_rad = angle_between(sc_closest_xyz-closest_point_rgt_xyz, sc_closest_xyz-too_target_xyz)
    off_point_angle_deg = off_point_angle_rad * 180 / np.pi
    direction = too_target_xyz - sc_closest_xyz
    start = sc_closest_xyz
    intersection_point, xy_dtm_target = get_gt_on_terrain(start, direction, dtm=dtm, dtm_data=dtm_data, fit_to_zero=False, disp=False)
    too_latlon = intersection_point[:2]
    time_utc_closest = datetime.datetime.fromtimestamp(np.interp(closest_point.lat, kml_df.lat, kml_df.dtstmp_utc))
    time_local_closest = time_utc_closest + timedelta_localtime
    time_utc_string = datetime.datetime.strftime(time_utc_closest,'%Y-%b-%dT%H:%M:%SZ')
    time_local_string = datetime.datetime.strftime(time_local_closest,'%Y-%b-%d %H:%M:%S') + (' %s' % local_time_zone)
    spacecraft_orientation = get_sc_orient(time_utc_closest)
    offset_distance = geodesic((too_latlon[0], too_latlon[1]), (too_submit_latlonelev[0], too_submit_latlonelev[1])).m
    offset_distance_approx = np.tan(off_point_angle_rad) * too_elev
    f = h5py.File(atl03_file, 'r')
    actual_offset_angle = np.mean(90 - 180*np.array(f['gt2l']['geolocation']['ref_elev'])/np.pi)
    beam_strength = {'left': 'weak', 'right': 'stron'} if spacecraft_orientation == 'forward' else {'left': 'strong', 'right': 'weak'}
    too_submit_utm = warp.transform({'init': 'epsg:4326'}, local_crs_meters, [too_submit_latlonelev[1]],[too_submit_latlonelev[0]])
    footprints_dict = get_footprints(dtm, closest_points_kml_df, too_submit_utm, spacecraft_orientation, 
                                     ground_track_buffer, ground_track_resolution, sc_closest_xyz, beam_strength, local_crs_meters)

    too_x_utm, too_y_utm = warp.transform({'init': 'epsg:4326'}, local_crs_meters, [too_latlon[1]], [too_latlon[0]])
    closest_x_utm, closest_y_utm = warp.transform({'init': 'epsg:4326'}, local_crs_meters, [closest_point.lon], [closest_point.lat])
    is2dict = {}
    min_dist_too_calc, min_dist_too_submit, min_dist_closest_is2 = 0, 0, 0
    for gtx in gtx_list:
        gtxs, ancillary, photon_data = read_atl03(atl03_file, geoid_h=False, gtxs_to_read=gtx)
        df_photon = get_signal(photon_data[gtx])
        ancillary = get_oa_url(df_photon, ancillary, gtx)
        df_grouped = get_pulseid_and_gt_latlon(df_photon[df_photon.is_signal].copy())
        df_elevs = get_along_track_elev(df_photon, df_grouped, xatc_resolution=1.0)
        df_elevs = interpolate_dtm(terrain_model_file, df_elevs)
        x_utm, y_utm = warp.transform({'init': 'epsg:4326'}, local_crs_meters, df_elevs.lon, df_elevs.lat)
        dist_too_calc = np.sqrt((np.array(x_utm)-too_x_utm[0])**2 + (np.array(y_utm)-too_y_utm[0])**2)
        dist_too_submit = np.sqrt((np.array(x_utm)-too_submit_utm[0][0])**2 + (np.array(y_utm)-too_submit_utm[1][0])**2)
        dist_closest_is2 = np.min(np.sqrt((np.array(x_utm)-closest_x_utm[0])**2 + (np.array(y_utm)-closest_y_utm[0])**2))
        df_elevs['x_utm'] = x_utm
        df_elevs['y_utm'] = y_utm
        df_elevs['dist_too_calc'] = dist_too_calc
        df_elevs['dist_too_submit'] = dist_too_submit
        df_elevs['dist_closest_is2'] = dist_closest_is2
        min_dist_too_calc += df_elevs.dist_too_calc.min() / 2
        min_dist_too_submit += df_elevs.dist_too_submit.min() / 2
        min_dist_closest_is2 += df_elevs.dist_closest_is2.min() / 2
        is2dict[gtx] = df_elevs
    dist_submit_closest = np.sqrt((too_submit_utm[0][0]-closest_x_utm[0])**2 + (too_submit_utm[1][0]-closest_y_utm[0])**2)
    dist_calc_closest = np.sqrt((too_x_utm[0]-closest_x_utm[0])**2 + (too_y_utm[0]-closest_y_utm[0])**2)
    if dist_submit_closest > min_dist_closest_is2:
        min_dist_too_submit = -min_dist_too_submit
    if dist_calc_closest > min_dist_closest_is2:
        min_dist_too_calc = -min_dist_too_calc

    print('\n___________________________________________________________________________________________________')
    print('KML used for RGT : %s' % kml_file_RGTpointingcycle)
    print('KML used for time of overpass : %s' % kml_file_thiscycle)
    print('ICESat-2 data granule : %s' % granule_name)
    print('Digital Terrain Model used : %s' % terrain_model_file)
    print('')
    print('Off-pointing angle estimate          : %.4f degrees' % off_point_angle_deg)
    print('Off-pointing angle actual from ATL03 : %.4f degrees' % actual_offset_angle)
    print('TOO calculated coordinates on topography : %11.7f, %12.7f (%7.2f m above WGS84)' % (too_latlon[0], too_latlon[1], too_elev))
    print('TOO submission coordinates               : %11.7f, %12.7f (%7.2f m above WGS84)' % tuple(too_submit_latlonelev))
    print('Offset distance between submission and calculated coordinates on topography : %6.1f m' % offset_distance)
    print('Offset distance estimated by tan(pointing_angle)*TOO_elevation              : %6.1f m' % offset_distance_approx)
    print('')
    print('Spacecraft elevation estimate used : %.3f km' % (spacecraft_elevation/1000))
    print('Spacecraft orientation from TechRefTable : %s ' % spacecraft_orientation)
    print('Estimated overpass time: %s (%s)' % (time_utc_string, time_local_string))
    print('Closest point on RGT : %.7f, %.7f' % (closest_point.lat, closest_point.lon))
    print('Off-pointing distance from sub_satellite point on ellipsoid : %.2f km' % (closest_point.dist/1000))
    print('')
    rel_direction = 'toward' if min_dist_too_submit<0 else 'away from'
    print('ATL03 data distance from submission point: %7.1f (%s the sub-satellite point)' % (min_dist_too_submit, rel_direction))
    rel_direction = 'toward' if min_dist_too_calc<0 else 'away from'
    print('ATL03 data distance from estimated  point: %7.1f (%s the sub-satellite point)' % (min_dist_too_calc, rel_direction))
    plot_over_dtm(dtm_viz_file, too_latlon, submission_point_dtm, footprints_dict, beam_strength)
    
    results = {'icesat2_kml_rgt_pointing_cycle': kml_file_RGTpointingcycle,
               'kml_file_this_cycle': kml_file_thiscycle,
               'icesat2_granule': granule_name,
               'terrain_model_file': terrain_model_file,
               'spacecraft_elevation': spacecraft_elevation,
               'spacecraft_spacecraft_orientation': spacecraft_orientation,
               'too_predicted_coordinates_on_topography': (too_latlon[0], too_latlon[1], too_elev),
               'too_submission_coordinates': tuple(too_submit_latlonelev),
               'off_point_angle_deg_predicted': off_point_angle_deg,
               'off_point_angle_deg_actual': actual_offset_angle,
               'distance_too_submission_vs_predicted': offset_distance,
               'distance_too_submission_vs_predicted_approx': offset_distance_approx,
               'time_estimate_utc': time_utc_string,
               'time_estimate_local': time_local_string,
               'closest_point_on_rgt': (closest_point.lat, closest_point.lon),
               'offpointing_distance_ellipsoid': closest_point.dist,
               'distance_is2data_from_submission': min_dist_too_submit,
               'distance_is2data_from_predicted': min_dist_too_calc
              }
    return results

In [157]:
# for KML download
download_dir_kmls_zip = 'data/IS2/kmls/all'
kml_url_list = ['https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle1_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle2_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle3_date_time_0.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle4_date_time_rev2_0.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle5_date_time_rev3.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle6_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle7_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle8_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle9_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle10_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle11_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2RGTscycle12datetime.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle13_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle14_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle15_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle16_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle17_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle18_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle19_date_time.zip',
                'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle20_date_time_provisional.zip',
               ]

# Download the Zip files
for url in kml_url_list:
    kml_zip_fn = url[url.rfind('/')+1:]
    kml_zip_path = download_dir_kmls_zip + '/' + kml_zip_fn
    print('downloading', kml_zip_path)
    r = requests.get(url, allow_redirects=True)
    open(kml_zip_path, 'wb').write(r.content)
    
# Unzip outputs
for z in os.listdir(download_dir_kmls_zip): 
    if z.endswith('.zip'): 
        print('--> extracting', zip_name)
        zip_name = download_dir_kmls_zip + "/" + z 
        zip_ref = zipfile.ZipFile(zip_name) 
        zip_ref.extractall(download_dir_kmls_zip) 
        zip_ref.close() 
        os.remove(zip_name)
        
# Clean up Outputs folder by removing individual granule folders 
print('Cleaning up outputs folder...', end=' ')
for root, dirs, files in os.walk(download_dir_kmls_zip, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), download_dir_kmls_zip)
        except OSError:
            pass
    for name in dirs:
        os.rmdir(os.path.join(root, name)) 
print(' --> DONE!')

downloading data/IS2/kmls/all/IS2_RGTs_cycle1_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle2_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle3_date_time_0.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle4_date_time_rev2_0.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle5_date_time_rev3.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle6_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle7_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle8_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle9_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle10_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle11_date_time.zip
downloading data/IS2/kmls/all/IS2RGTscycle12datetime.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle13_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle14_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle15_date_time.zip
downloading data/IS2/kmls/all/IS2_RGTs_cycle16_date_time.zip
downloading data/IS2/km

OSError: [Errno 66] Directory not empty: 'data/IS2/kmls/all/IS2_RGTs_cycle1_date_time'

In [180]:
# Clean up Outputs folder by removing individual granule folders 
print('Cleaning up outputs folder...', end=' ')
for root, dirs, files in os.walk(download_dir_kmls_zip, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), download_dir_kmls_zip)
        except OSError:
            pass
    for name in dirs:
        # shutil.rmtree(name, ignore_errors=True)
        print(root+'/',name)
print(' --> DONE!')

Cleaning up outputs folder... data/IS2/kmls/all/ IS2_RGTs_cycle1_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle13_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle6_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle4_date_time_rev2
data/IS2/kmls/all/ IS2_RGTs_cycle12_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle5_date_time
data/IS2/kmls/all/ __MACOSX
data/IS2/kmls/all/ IS2_RGTs_cycle8_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle3_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle11_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle9_date_time
data/IS2/kmls/all/ cycle2_rgts_date_time
data/IS2/kmls/all/ IS2_RGTs_cycle10_date_time
 --> DONE!


In [174]:
kml_filelist = [download_dir_kmls_zip+'/'+f for f in os.listdir(download_dir_kmls_zip) \
                if os.path.isfile(os.path.join(download_dir_kmls_zip, f))]
for i in np.arange(1,21):
    cycle_search = '_cycle%i_' % i
    cycle_filelist = [f for f in kml_filelist if ((cycle_search in f) & (f[len(download_dir_kmls_zip)+1] != '.'))]
    print(cycle_search, len(cycle_filelist))

_cycle1_ 0
_cycle2_ 1387
_cycle3_ 1387
_cycle4_ 1221
_cycle5_ 1387
_cycle6_ 1387
_cycle7_ 1387
_cycle8_ 1387
_cycle9_ 1387
_cycle10_ 1387
_cycle11_ 1387
_cycle12_ 1387
_cycle13_ 1387
_cycle14_ 1387
_cycle15_ 1387
_cycle16_ 1387
_cycle17_ 1387
_cycle18_ 1387
_cycle19_ 1387
_cycle20_ 1387


In [ ]:
# for ICESat-2 data download
datelatlon = [['2022-02-04', 32.986318, -106.420443],
              ['2022-05-06', 32.986318, -106.420443],
              ['2022-05-25', 32.986317, -106.420565],
              ['2022-06-04', 32.982232, -106.420101],
             ]


output_dir_atl03 = 'data/IS2/atl03/wsmr/'
uid, pwd, email = getedcreds()
x_query_dist_buffer = 50000 # so that we actually find the granule
for date, lat, lon in datelatlon:
    crs_utm = get_utm_crs_from_latlon(lat, lon)
    distance_buffer = np.max(np.abs(np.array(kwargs['ground_track_buffer'])))
    center_x, center_y = warp.transform({'init': 'epsg:4326'}, crs_utm, [lon], [lat])
    xl = [center_x[0] - x_query_dist_buffer, center_x[0] + x_query_dist_buffer]
    yl = [center_y[0] - distance_buffer, center_y[0] + distance_buffer]
    bbx = [xl[i] for i in [0, 1, 1, 0]]
    bby = [yl[i] for i in [0, 0, 1, 1]]
    bblon, bblat = warp.transform(crs_utm, {'init': 'epsg:4326'}, bbx, bby)
    lonmin = np.min(bblon)
    latmin = np.min(bblat)
    lonmax = np.max(bblon)
    latmax = np.max(bblat)
    boundbox = [lonmin, latmin, lonmax, latmax]
    print(date, boundbox)
    download_is2(start_date=date, end_date=date, uid=uid, pwd=pwd, bbox=boundbox, output_dir=output_dir_atl03, email=email)

In [158]:
output_dir_atl03 = 'data/IS2/atl03/wsmr/'

In [105]:
kwargs = {'spacecraft_elevation': 484e3, # meters (roughly)
          'terrain_model_file': 'data/LiDAR/WSMR/output_USGS10m.tif',
          'dtm_viz_file': 'data/LiDAR/WSMR/viz.USGS10m_hillshade-color.tif',
          'local_time_zone': 'America/Albuquerque',
          'ground_track_buffer': (-1000, 1000), # meters
          'ground_track_resolution': 1000
         }

In [141]:
resultlist = []
dates = np.array(datelatlon)[:,0]
atl03filelist = [output_dir_atl03+f for f in os.listdir(output_dir_atl03) if os.path.isfile(os.path.join(output_dir_atl03, f))]
rgtpointingcycle = '_cycle14_'

for i, date in enumerate(dates):
    print('_________________________________________________________________________________________________')
    print(date)
    datecode = date[:4] + date[5:7] + date[8:]
    files = [f for f in atl03filelist if datecode in f]
    
    for fn in files:
        granule_name = fn[fn.find('ATL03_'):]
        track_string = granule_name[21:25]
        kml_search_track = 'RGT_' + track_string
        dtime = datetime.datetime.strptime(date, '%Y-%m-%d')
        date_kml_string = datetime.datetime.strftime(dtime, '%d-%b-%Y')
        print(granule_name)
        print('track:', track_string)
        print(date_kml_string)
        print()

_________________________________________________________________________________________________
2022-02-04
ATL03_20220204125022_06761402_005_01.h5
track: 0676
04-Feb-2022

_________________________________________________________________________________________________
2022-05-06
ATL03_20220506083015_06761502_005_02.h5
track: 0676
06-May-2022

_________________________________________________________________________________________________
2022-05-25
ATL03_20220525193847_09731506_005_02.h5
track: 0973
25-May-2022

_________________________________________________________________________________________________
2022-06-04
ATL03_20220604070622_11181502_005_02.h5
track: 1118
04-Jun-2022



In [93]:
too_submit_latlonelev = np.array((37.90704, -107.7263, 0.0))
kml_file_thiscycle = 'data/IS2/kmls/IS2_RGT_0234_cycle11_09-Apr-2021.kml' # includes veg off-pointing -> use RGT pointing KML for geometry instead
kml_file_RGTpointingcycle = 'data/IS2/kmls/IS2_RGT_0234_cycle12_08-Jul-2021.kml' # cycle 12 points to RGT, so represents sub-satellite point
atl03_file = 'data/IS2/atl03/processed_ATL03_20210409031439_02341102_005_01.h5'

In [96]:
res = check_too(too_submit_latlonelev, kml_file_thiscycle, kml_file_RGTpointingcycle, atl03_file, **kwargs)

Calculating laser footprint track on the topography (left side, weak beam)
---> DONE!                                                                        
Calculating laser footprint track on the topography (right side, stron beam)
---> DONE!                                                                        
  reading in data/IS2/atl03/processed_ATL03_20210409031439_02341102_005_01.h5
  reading in beam: gt2l  --> done.
http://openaltimetry.org/data/api/icesat2/atl03?date=2021-04-09&minx=-107.7321567&miny=37.8570207&maxx=-107.7198244&maxy=37.9569999&trackId=234&beamName=gt2l&outputFormat=json
  reading in data/IS2/atl03/processed_ATL03_20210409031439_02341102_005_01.h5
  reading in beam: gt2r  --> done.
http://openaltimetry.org/data/api/icesat2/atl03?date=2021-04-09&minx=-107.7314329&miny=37.8570925&maxx=-107.7187759&maxy=37.9570750&trackId=234&beamName=gt2r&outputFormat=json

___________________________________________________________________________________________________
KML

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …